## Setup
Initialize connection to the hubs

In [ ]:
from pybricksdev.connections import BLEPUPConnection, EV3Connection
from asyncio import gather, sleep

class BLEPUPConnectionExtra(BLEPUPConnection):
    def line_handler(self, line):
        try:
            self.line_buffer.append(line)
        except:
            self.line_buffer = []
        super().line_handler(line)

try:
    switch, train
except:
    switch = BLEPUPConnectionExtra()
    await switch.connect('90:84:2B:01:14:27')
    
    train = BLEPUPConnectionExtra()
    await train.connect('90:84:2B:04:C3:81')

## Train script
Output color sensor detection to variable

In [ ]:
%%file build/train.py

from pybricks.hubs import CityHub
from pybricks.pupdevices import ColorDistanceSensor
from pybricks.parameters import Port, Color
from pybricks.tools import wait, StopWatch

hub = CityHub()
sensor = ColorDistanceSensor(Port.B)
speed = 0

#wait(2000)

while True:
    color = sensor.color()
    #print(color)
    if color == Color.RED:
        hub.light.on(Color.RED)
        speed = 0
    elif color == Color.YELLOW:
        hub.light.on(Color.YELLOW)
        speed = 1
    elif color == Color.WHITE:
        hub.light.on(Color.WHITE)
        speed = 2
    print("__data__{0}".format(speed))
    wait(500)

## Switch script
Evaluate value recieved from train and act on it

In [ ]:
%%file build/switch.py

from pybricks.hubs import CityHub
from pybricks.pupdevices import ColorDistanceSensor, DCMotor
from pybricks.parameters import Port, Color
from pybricks.tools import wait, StopWatch

hub = CityHub()
motor = DCMotor(Port.A)
speed = 0

#wait(2000)

while True:
    motor.stop()
    speed = eval(input())
    if speed == 0:
        hub.light.on(Color.RED)
    elif speed == 1:
        hub.light.on(Color.YELLOW)
        motor.dc(100)
        wait(200)
        motor.stop()
    elif speed == 2:
        hub.light.on(Color.WHITE)
        motor.dc(-100)
        wait(400)
        motor.stop()
    wait(500)

## PC script
Reads value from Train and sends it to all other hubs

In [ ]:
# We can do anything we want with the data, like displaying it.
# In this example, we just send everything received from one hub
# to all the other hubs.

DATA = b'__data__'

async def forwarder(*hubs):
    await sleep(2)
    
    while any(hub.state == hub.RUNNING for hub in hubs):
        for hub in hubs:
            while len(hub.line_buffer) > 0:
                line = hub.line_buffer.pop(0)
                if DATA in line:
                    for remote in hubs:
                        if remote != hub:  
                            await remote.write(line[len(DATA):] + b'\r\n')
        await sleep(0.2)

## Run everything

In [ ]:
await gather(
    switch.run('build/switch.py'),
    train.run('build/train.py'),
    forwarder(switch)
)